In [17]:
# import intake
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as cm
# import cmocean.cm as cmo
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
# import gribscan
import glob
import intake
import dask
# dask.config.set(array_slicing_split_large_chunks=True)
# dask.config.set({"array.slicing.split_large_chunks": True}) 

from numcodecs import Blosc

In [18]:
dask.__version__

'2023.3.0'

In [19]:
cat = intake.open_catalog("../../catalog.yaml")

In [20]:
for model, vm in cat.items():
    for exp, _ in vm.items():
        print(model + '\t' + exp + '\t' + cat[model][exp].description)
        if exp != "grids":
            for k in cat[model][exp]:
                print('\t' + '- ' + k + '\t' + cat[model][exp].walk()[k]._description)  # pylint: disable=W0212
    print()

FESOM	tco2559-ng5-cycle3	4.5km experiment, coupuled with IFS
	- 2d_elements_monthly	
	- 2d_vertices_daily	
	- 2d_vertices_hourly	
	- 3d_elements_nz1-upper_3hourly	
	- 3d_elements_nz1_daily	
	- 3d_elements_nz_daily	
	- 3d_vertices_nz-upper_3hourly	
	- 3d_vertices_nz1-upper_3hourly	
	- 3d_vertices_nz1_daily	
	- 3d_vertices_nz_daily	
	- elem_grid	
	- node_grid	
FESOM	tco2559-ng5-cycle3_interpolated	4.5km experiment, coupuled with IFS
	- 025_degree_monthly	
	- 1_degree_monthly	
FESOM	tco2559-ng5-cycle3_zarr	4.5km experiment, coupuled with IFS
	- 3d_vertices_nz1_daily	
	- 3d_vertices_nz1_monthly	
	- elem_grid	
	- node_grid	



In [21]:
run = cat.FESOM['tco2559-ng5-cycle3_zarr']

In [6]:
from dask.distributed import Client
import dask
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})
dask.config.set({'temporary_directory': '/scratch/a/a270088/dask/'})
# client = Client(n_workers=5, threads_per_worker=1, memory_limit='50GB')
client = Client(n_workers=20, threads_per_worker=1, memory_limit='10GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Workers: 20
Total threads: 20,Total memory: 186.26 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42835,Workers: 20
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Total threads: 20
Started: Just now,Total memory: 186.26 GiB
Comm: tcp://127.0.0.1:44447,Total threads: 1
Dashboard: /user/a270088/levante-spawner-preset//proxy/43579/status,Memory: 9.31 GiB
Nanny: tcp://127.0.0.1:32813,


## Compute TS monthly means

In [22]:
# with dask.config.set(array_slicing_split_large_chunks=True):
data = run['3d_vertices_nz1_daily'].to_dask()

In [23]:
data

<xarray.Dataset>
Dimensions:  (nz1: 69, time: 712, nod2: 7402886)
Coordinates:
  * nz1      (nz1) float64 2.5 7.5 12.5 17.5 ... 5.525e+03 5.825e+03 6.175e+03
  * time     (time) datetime64[ns] 2020-01-20T23:56:00 ... 2021-12-31T23:56:00
Dimensions without coordinates: nod2
Data variables:
    salt     (time, nz1, nod2) float32 dask.array<chunksize=(12, 5, 1000000), meta=np.ndarray>
    temp     (time, nz1, nod2) float32 dask.array<chunksize=(12, 5, 1000000), meta=np.ndarray>

In [16]:
for year in range(2020, 2022):
    print(year)
    for vari in ['salt', 'temp']:
        print(f"variable: {vari}, year: {year}")
        vari_mm = data[vari].sel(time=str(year)).resample(time='1M').mean(dim='time')
        vari_mm.to_netcdf(f'/work/bm1344/AWI/Cycle3/test/monthly_means/{vari}/{vari}_{str(year)}.nc')

2020
variable: salt, year: 2020
variable: temp, year: 2020
2021
variable: salt, year: 2021
variable: temp, year: 2021
